## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [1]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Recall that we need to check for missing data.

In [3]:
def missingness_summary(df, print_log=False, sort='none'):
    summary = df.apply(lambda x: x.isna().sum() / x.shape[0])
    
    if sort == 'none':
        summary = summary
    elif sort == 'ascending':
        summary = summary.sort_values()
    elif sort == 'descending':
        summary = summary.sort_values(ascending=False)
    else:
        print('Invalid value for sort parameter.')
    
    if print_log:
        print(summary)
        
    return summary

In [4]:
# answer below:
missingness_summary(wine)


fixed.acidity           0.0
volatile.acidity        0.0
citric.acid             0.0
residual.sugar          0.0
chlorides               0.0
free.sulfur.dioxide     0.0
total.sulfur.dioxide    0.0
density                 0.0
pH                      0.0
sulphates               0.0
alcohol                 0.0
quality                 0.0
dtype: float64

Convert quality to a binary variable, with the dividing line between 5 and 6.

In [14]:
# answer below
wine.quality.value_counts()
def bin_split(value, split):
  if value < split:
    return 0
  else:
    return 1


bin_quality = wine['quality'].apply(lambda x: bin_split(x, 6))
clean_wine = wine.drop('quality', axis=1)
clean_wine = pd.concat([clean_wine, bin_quality], axis=1)
clean_wine

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
2,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0
3,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0
4,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1
5,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1595,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,0
1596,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,1
1597,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,1
1598,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,0


Create a train test split with 20% of the data in the test subsample.

In [15]:
# answer below:
from sklearn.model_selection import train_test_split
X = clean_wine.drop('quality', axis=1)
y = clean_wine['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


Scale only the independent variables using the minmax scaler.

In [25]:
# answer below:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_fitted = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
X_test_fitted = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)



Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [127]:
# answer below:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
fit = knn.fit(X_train, y_train)
unscaled_train_pred = fit.predict(X_train)
unscaled_test_pred = fit.predict(X_test)

train_score = accuracy_score(unscaled_train_pred, y_train)
test_score = accuracy_score(unscaled_test_pred, y_test)
print('unscaled train_score:', train_score)
print('unscaled test_score:', test_score)

knn = KNeighborsClassifier(n_neighbors=5)
fit = knn.fit(X_train_fitted, y_train)
scaled_train_pred = fit.predict(X_train_fitted)
scaled_test_pred = fit.predict(X_test_fitted)

train_score = accuracy_score(scaled_train_pred, y_train)
test_score = accuracy_score(scaled_test_pred, y_test)
print('scaled train_score:', train_score)
print('scaled test_score:', test_score)


unscaled train_score: 0.7771696637998436
unscaled test_score: 0.659375
scaled train_score: 0.8037529319781079
scaled test_score: 0.728125


When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [126]:
def inv_sqrt_1(dist):
  for i in range(len(dist)):
    for j in range(len(dist[i])):
      if dist[i][j] != 0:
        dist[i][j] = math.sqrt(dist[i][j])
      else:
        dist[i][j] = math.inf
  return dist
  
    

In [123]:
def inv_sqrt(dist):
  # if user attempts to classify a point that was zero distance from one
  # or more training points, those training points are weighted as 1.0
  # and the other points as 0.0
  if dist.dtype is np.dtype(object):
      for point_dist_i, point_dist in enumerate(dist):
          # check if point_dist is iterable
          # (ex: RadiusNeighborClassifier.predict may set an element of
          # dist to 1e-6 to represent an 'outlier')
          if hasattr(point_dist, '__contains__') and 0. in point_dist:
              dist[point_dist_i] = point_dist == 0.
          else:
              dist[point_dist_i] = 1. / point_dist
              dist[point_dist_i] = dist[point_dist_i].apply(lambda x: math.sqrt(x))
  else:
      with np.errstate(divide='ignore'):
          dist = 1. / dist
      inf_mask = np.isinf(dist)
      inf_row = np.any(inf_mask, axis=1)
      dist[inf_row] = inf_mask[inf_row]
  return dist

In [125]:
knn = KNeighborsClassifier(n_neighbors=5, weights=inv_sqrt_1)
fit = knn.fit(X_train_fitted, y_train)
scaled_train_pred = fit.predict(X_train_fitted)
scaled_test_pred = fit.predict(X_test_fitted)

train_score = accuracy_score(scaled_train_pred, y_train)
test_score = accuracy_score(scaled_test_pred, y_test)
print('scaled train_score:', train_score)
print('scaled test_score:', test_score)

scaled train_score: 0.6489444878811571
scaled test_score: 0.696875
